In [2]:
import pandas as pd

In [6]:
df_nys_data = pd.read_csv("https://s3.amazonaws.com/tmp-gfx-public-data/goodstats20230210/Index_Crimes_by_County_and_Agency__Beginning_1990.csv")
df_nys_estimation = pd.read_csv("https://s3.amazonaws.com/tmp-gfx-public-data/goodstats20230210/Indicator_Tables_no_supp_no_LEOKA_88.csv")

In [7]:
df_nys_data_summary = df_nys_data[(df_nys_data["Year"] == 2021) &\
            (df_nys_data["Agency"] == "County Total")][["Index Total", "Violent Total", "Property Total",\
             "Murder", "Rape", "Robbery", "Aggravated Assault",\
             "Burglary", "Larceny", "Motor Vehicle Theft"]].sum(axis=0)\
             .reset_index() \
             .rename(columns = {
                 "index": "crime",
                 0: "nys_total"
             })

In [8]:
df_nys_data_summary

,crime,nys_total
0,Index Total,343357.0
1,Violent Total,76229.0
2,Property Total,267128.0
3,Murder,880.0
4,Rape,6213.0
5,Robbery,18050.0
6,Aggravated Assault,51086.0
7,Burglary,27129.0
8,Larceny,218129.0
9,Motor Vehicle Theft,21870.0


In [9]:
df_nys_estimation.sample(10)

,indicator_name,estimate,estimate_unweighted,estimate_geographic_location,estimate_type,estimate_type_num,estimate_type_detail_percentage,estimate_type_detail_rate,estimate_domain_1,estimate_domain_2,...,der_rrmse_30_agency_10,prop_30_10,der_perm_group_unsuppression_flag,der_perm_group_suppression_flag,population_estimate,time_series_start_year,full_table,der_variable_name,PERMUTATION_NUMBER,PRB_ACTUAL
7998,Revised Rape,310.456140,101.000000,Age: 25-34 State New York,count,1,NaN,NaN,Injury,No,...,0,0.552239,0,0,-9.0,2021,Table3a-Person Victims,t_3a_10_74_15,7088,25.277678
8969,Property Crime,5637.372807,1787.000000,Age: 25-34 State New York,count,1,NaN,NaN,Arrestee sex and race,Male,...,0,0.552239,0,0,-9.0,2021,Table5a-Arrestees Arrest Code,t_5a_6_24_31,7088,-10.897805
14048,Kidnapping/Abduction,100.000000,100.000000,Sex: Female State New York,percentage,2,Arrestee Level,NaN,Arrestee armed,No,...,0,0.582090,0,0,-9.0,2021,Table4a-Arrestees,t_4a_9_52_7,11088,-14.716826
4896,Robbery,72.687963,78.483607,State New York,percentage,2,Arrestee Level,NaN,Multiple arrest indicator,Not applicable,...,0,0.550296,0,0,-9.0,2021,Table5a-Arrestees Arrest Code,t_5a_8_51_26,88,-42.520757
2791,Fondling,13.130139,13.510520,State New York,percentage,2,Victim Level,NaN,Victim Age,18-24,...,0,0.550296,0,0,-9.0,2021,Table3a-Person Victims,t_3a_2_10_13,88,-30.233866
2184,Drug/Narcotic Offenses,3134.166667,781.000000,State New York,count,1,NaN,NaN,Population group,"Cities and counties 10,000-24,999",...,0,0.550296,0,0,-9.0,2021,Table2c-Society Offenses,t_2c_5_29_3,88,-3.269273
11603,NIBRS crimes against property (Total),31389.627193,10260.000000,Sex: Male State New York,count,1,NaN,NaN,Arrest count,NaN,...,0,0.471429,0,0,-9.0,2021,Table4a-Arrestees,t_4a_1_1_15,10088,-21.727122
18755,"Sex Offenses, Nonforcible",80.513158,23.000000,Age: 18+ State New York,count,1,NaN,NaN,Juvenile disposition,Not applicable,...,0,0.450704,0,0,-9.0,2021,Table5a-Arrestees Arrest Code,t_5a_7_47_14,17088,-17.185940
11048,"35A - Cocaine/Crack Cocaine (A,B) - P (Possess...",904.008772,277.000000,Age: 35-64 State New York,count,1,NaN,NaN,Total Arrests per Drug Category,NaN,...,0,0.507246,0,0,-9.0,2021,TableDM7 - Drug Count By Arrestee,t_DM7_2_2_4,8088,7.722102
10131,NIBRS crimes against property (Total),70.592224,67.137981,Age: 35-64 State New York,percentage,2,Arrestee Level subset to male,NaN,Arrestee sex and race Male,White,...,0,0.507246,0,0,-9.0,2021,Table4a-Arrestees,t_4a_6_25_15,8088,222.477194


In [10]:
## extract index crimes
df_nys_estimation_index_crime = df_nys_estimation[df_nys_estimation["indicator_name"].isin(["Violent Crime", "Property Crime",\
                                          "Murder and Non-negligent Manslaughter", "Rape", "Robbery", "Aggravated Assault",\
                                          "Burglary/B&E", "Larceny/Theft Offenses", "Motor Vehicle Theft"])]

In [11]:
## extract crime estimation summaries

df_nys_estimation_index_crime_summary = df_nys_estimation_index_crime[(df_nys_estimation_index_crime["estimate_type"] == "count") &\
                              (df_nys_estimation_index_crime["estimate_domain_1"] == "Offense count")]\
                              [["indicator_name", "estimate", "estimate_unweighted", "estimate_upper_bound", "estimate_lower_bound"]]
                            
df_nys_estimation_index_crime_summary["indicator_name"] = df_nys_estimation_index_crime_summary["indicator_name"].replace({
                                  "Violent Crime": "Violent Total",
                                  "Burglary/B&E": "Burglary",
                                  "Larceny/Theft Offenses": "Larceny",
                                  "Property Crime": "Property Total"
                              })

df_nys_estimation_index_crime_summary = df_nys_estimation_index_crime_summary.rename(columns = {
    "indicator_name": "crime"
})


In [12]:
## Merge NYS data and the FBI's estimations
df = pd.merge(df_nys_estimation_index_crime_summary, df_nys_data_summary, on="crime", how="outer")

In [13]:
df = df.set_index("crime")
df.loc["Index Total", "estimate"] = df.loc["Violent Total", "estimate"] + df.loc["Property Total", "estimate"]
df.loc["Index Total", "estimate_unweighted"] = df.loc["Violent Total", "estimate_unweighted"] + df.loc["Property Total", "estimate_unweighted"]
df["unweighted_pct"] = df["estimate_unweighted"] / df["nys_total"]
df["estimate/total"] = df["estimate"] / df["nys_total"]

In [14]:
df

,estimate,estimate_unweighted,estimate_upper_bound,estimate_lower_bound,nys_total,unweighted_pct,estimate/total
crime,,,,,,,
Rape,1420.530702,432.0,1958.666098,882.395306,6213.0,0.069532,0.228638
Violent Total,20202.815789,7464.0,31513.366511,8892.265068,76229.0,0.097915,0.265028
Burglary,15523.881579,5406.0,24573.598267,6474.164891,27129.0,0.199270,0.572225
Larceny,114465.517544,39977.0,180474.092856,48456.942232,218129.0,0.183272,0.524761
Property Total,140832.311404,49595.0,223183.227308,58481.395499,267128.0,0.185660,0.527209
Index Total,161035.127193,57059.0,NaN,NaN,343357.0,0.166180,0.469002
Murder,NaN,NaN,NaN,NaN,880.0,NaN,NaN
Robbery,NaN,NaN,NaN,NaN,18050.0,NaN,NaN
Aggravated Assault,NaN,NaN,NaN,NaN,51086.0,NaN,NaN


In [16]:
#df.sort_values("estimate", ascending = False).to_csv('D:/temp/Columbia/newsroom/Marshall/NIBRS_estimation.csv')